# Part 1
Fill in the partially complete lines of code in the file <br>
temporal_context_model_empty.m to make a temporal context model of memory <br>
retrieval that retrieves about 7 items from an encoded list efficiently. 


In [1]:
import numpy as np
from drawFromADist import drawFromADist

Writing the code in function so that we can find mean number of retrievals

In [2]:

def model_1(item_occurance, random_world_m = True, beta_param=0.99):

    # Trying to replicate matlab code
    """the temporal context model assumes that the past becomes increasingly<br>
    dissimilar to the future, so that memories become harder to retrieve the<br>
    farther away in the past they are
    """
    
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    """we are going to model the world as a set of N continuous-valued features.<br>
    we will model observations of states of the world as samples from N<br>
    Gaussians with time-varying means and fixed variance. For simplicity,<br>
    assume that agents change nothing in the world.
    """
    schedule_list = np.sort(np.ceil(item_occurance)).reshape(-1, 1)
    item_list = np.array(range(N_ITEMS)).reshape(-1,1)
    schedule = np.concatenate((schedule_list, item_list), axis=1)  # https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html
    schedule_load = ENCODING_TIME/(np.median(np.diff(schedule[:,0])))
    encoding = np.zeros((N_ITEMS, N_WORLD_FEATURES + 1))
    if random_world_m:
        # Generate random world 
        world_m = np.random.randint(low=1, high=5, size=N_WORLD_FEATURES)
    else:
        world_m = np.array([1, 2, 1, 2, 3]) 
    world_var = 1
    delta = 0.05        # This parameter changes the world context
    #beta_param = 0.9  # This parameter will specify the ratio of drift, i.e how much of previous context do we preserve
    # Keeping beta param 0.99 because delta is constant and having small value of beta will shrink the world_var towards 0.

    m = 0 # Changing index of m from 1 to 0, because matlab indexing starts from 1 and python from 0

    # simulating encoding
    for time in range(1, ENCODING_TIME+1):
        world_m = beta_param*world_m + delta 
        # In slides, world drift is written as, ct = beta*c{t-1} + ct_IN
        world = np.random.normal(loc=world_m, scale=world_var)   # https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html
        #  % any item I want to encode in memory, I encode in association with the
        #  % state of the world at that time.

        if m < N_ITEMS: # m will run from 0 to N_ITEMS - 1 times, because python indexes from 0 instead of 1
            if time == schedule[m][0]:
                encoding[m] = np.append(world, m)  # We store the world state and element id as encoding
                m += 1


    out=np.zeros(TEST_TIME)

    # % simulating retrieval using SAM, but with a bijective image-item mapping
    while time < ENCODING_TIME + TEST_TIME:
        # % the state of the world is the retrieval cue
        world_m = beta_param*world_m + delta
        world = np.random.normal(loc=world_m, scale=world_var)  

        soa=np.zeros(N_ITEMS)
        current_world=np.zeros(N_WORLD_FEATURES+1)
        for m in range(N_ITEMS):
            current_world=  np.append(world, m)
            soa[m]=np.dot(encoding[m],current_world.T) 
        
        #NORMALIZE
        soa=soa/np.sqrt(np.sum(soa**2)) 
        # REF: https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
        
        out[time - ENCODING_TIME] = np.nonzero(drawFromADist(soa))[0][0] # Get the non zero element 
        # REF: https://www.geeksforgeeks.org/numpy-nonzero-in-python/

        time = time + 1
    # % humans can retrieve about 7 items effectively from memory. get this model
    # % to behave like humans

    return len(np.unique(out))

First i will try various values of beta and delta to see which works best

In [3]:
beta_list = [0.001, 0.01, 0.1, 0.5, 0.9, 0.95, 0.99]
item_occurance = np.random.random(10) * 500
for beta in beta_list:
    out = []
    num_of_trials = 20
    for i in range(num_of_trials):

        success = model_1(item_occurance, random_world_m=False ,beta_param=beta)
        out.append(success)
    print(f"Beta:{beta}, success:{np.mean(out)}")


Beta:0.001, success:6.5
Beta:0.01, success:6.3
Beta:0.1, success:6.2
Beta:0.5, success:6.15
Beta:0.9, success:6.55
Beta:0.95, success:6.9
Beta:0.99, success:8.35


Best beta value is 0.99, so we will use that value. (NOTE: I got value of 0.99 as best but it might change depending on various runs)

In [4]:
out = []
num_of_trials = 25
for i in range(num_of_trials):
    item_occurance = np.random.random(10) * 500
    out.append(model_1(item_occurance, random_world_m=False, beta_param=0.99))
print(f"Mean of number of items retrieved over {num_of_trials} is :{np.mean(out)}")

Mean of number of items retrieved over 25 is :8.0
